In [1]:
from metasmith.models.solver import _solve_by_bounded_dfs
from metasmith.models.solver import Namespace, Transform, Endpoint, Result

In [5]:
def _print_sol(plan: Result):
    for i, step in enumerate(plan.dependency_plan):
        print(f"step {i+1}")
        print(step.transform)
        print("used")
        for x in step.used:
            print(" ", x)
        print("produced")
        for x in step.produced:
            print(" ", x)
        # if len(step.deleted) > 0:
        #     print("deleted")
        #     for x in step.deleted:
        #         print(" ", x)
        print()

def _set(s: str):
    return set(s.split(", "))

In [3]:
transforms = []

t = Transform() # assembly <-> bins
t.AddRequirement(properties={"assembly"})
t.AddProduct(properties={"bins"})
transforms.append(t)
t = Transform()
t.AddRequirement(properties={"bins"})
t.AddProduct(properties={"assembly"})
transforms.append(t)

t = Transform() # bins <-> tax
t.AddRequirement(properties={"bins"})
t.AddProduct(properties={"tax"})
transforms.append(t)
t = Transform()
t.AddRequirement(properties={"tax"})
t.AddProduct(properties={"bins"})
transforms.append(t)

t = Transform() # bins <-> contigs
t.AddRequirement(properties={"bins"})
t.AddProduct(properties={"contigs"})
transforms.append(t)
t = Transform()
t.AddRequirement(properties={"contigs"})
t.AddProduct(properties={"bins"})
transforms.append(t)

t = Transform() # contigs <-> ORFs
t.AddRequirement(properties={"contigs"})
t.AddProduct(properties={"ORFs"})
transforms.append(t)
t = Transform()
t.AddRequirement(properties={"ORFs"})
t.AddProduct(properties={"contigs"})
transforms.append(t)

t = Transform() # ORFs <-> annotation
t.AddRequirement(properties={"ORFs"})
t.AddProduct(properties={"annotation"})
transforms.append(t)
t = Transform() 
t.AddRequirement(properties={"annotation"})
t.AddProduct(properties={"ORFs"})
transforms.append(t)


have = [
    Endpoint(properties={"annotation"}),
]

target = Transform()
tax = target.AddRequirement(properties={"tax"})
target.AddRequirement(properties={"annotation"}, parents={tax})

plan = _solve_by_bounded_dfs(have, target, transforms)[0]
# plan = Solve(have, target, transforms)[0]
_print_sol(plan)

step 1
{annotation}->{ORFs}
used
  <["annotation"]:aQ8go>
produced
  <["ORFs"]:YX1WH>

step 2
{ORFs}->{contigs}
used
  <["ORFs"]:YX1WH>
produced
  <["contigs"]:VrA0N>

step 3
{contigs}->{bins}
used
  <["contigs"]:VrA0N>
produced
  <["bins"]:HkGJt>

step 4
{bins}->{tax}
used
  <["bins"]:HkGJt>
produced
  <["tax"]:IxqNN>

step 5
{tax}->{bins}
used
  <["tax"]:IxqNN>
produced
  <["bins"]:Mv3lz>

step 6
{bins}->{contigs}
used
  <["bins"]:Mv3lz>
produced
  <["contigs"]:hlk0q>

step 7
{contigs}->{ORFs}
used
  <["contigs"]:hlk0q>
produced
  <["ORFs"]:CxGGz>

step 8
{ORFs}->{annotation}
used
  <["ORFs"]:CxGGz>
produced
  <["annotation"]:56DV2>



In [17]:
transforms = []

t = Transform()
t.AddRequirement(properties=_set("reads"))
t.AddRequirement(properties=_set("container, provides=1"))
t.AddProduct(properties=_set("annable, taxable"))
transforms.append(t)

t = Transform()
t.AddRequirement(properties=_set("annable"))
t.AddRequirement(properties=_set("container, provides=2"))
t.AddRequirement(properties=_set("ref, provides=2"))
t.AddProduct(properties=_set("ann"))
transforms.append(t)

# t = Transform()
# t.AddRequirement(properties=_set("ann"))
# t.AddProduct(properties=_set("annable"))
# transforms.append(t)

t = Transform()
t.AddRequirement(properties=_set("taxable"))
t.AddProduct(properties=_set("tax"))
transforms.append(t)

t = Transform()
d_parent = t.AddRequirement(properties=_set("annable, taxable"))
d_ann = t.AddRequirement(properties=_set("ann"), parents={d_parent})
d_tax = t.AddRequirement(properties=_set("tax"), parents={d_parent})
t.AddProduct(properties=_set("sum"))
transforms.append(t)

# M, N = 2, 1
# M, N = 2, 2
# M, N = 5, 5
# M, N = 50, 2
# M, N = 64, 64
# M, N = 128, 128
M, N = 256, 256
haves = [Endpoint(properties=_set(f"{i+1}, reads")) for i in range(M)]
haves += [Endpoint(properties=_set(f"container, provides={i+1}")) for i in range(3)]
haves += [Endpoint(properties=_set(f"ref, provides={i+1}")) for i in range(3)]

target = Transform()
# for e in haves[-N:]:
for e in haves[:N]:
    de = target.AddRequirement(properties=e.properties)
    target.AddRequirement(properties=_set("ann"), parents={de})
    # target.AddRequirement(properties=_set("sum"))

print("Start")
print(len(haves), len(target.requires))
# %prun r = Solve(haves, target, transforms)
%prun solutions = _solve_by_bounded_dfs(haves, target, transforms, _debug=True)
# f"input size [{N}], states checked [{r.steps}], {r.message}, {len(target.requires)}"

Start
262 512
 

         9933148 function calls (9449196 primitive calls) in 3.421 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   339508    0.465    0.000    0.853    0.000 solver.py:160(Pack)
   339512    0.322    0.000    0.322    0.000 encoder.py:205(iterencode)
   339512    0.299    0.000    1.099    0.000 __init__.py:183(dumps)
   339512    0.275    0.000    0.718    0.000 encoder.py:183(encode)
   339508    0.231    0.000    2.306    0.000 solver.py:69(__str__)
   236213    0.150    0.000    2.738    0.000 solver.py:370(<genexpr>)
   402978    0.142    0.000    2.099    0.000 solver.py:72(__repr__)
   952732    0.129    0.000    0.129    0.000 {method 'startswith' of 'str' objects}
   339508    0.123    0.000    1.222    0.000 solver.py:132(_json_dumps)
559537/80218    0.123    0.000    2.801    0.000 {method 'join' of 'str' objects}
   613223    0.121    0.000    0.200    0.000 solver.py:166(_unlist)
   392704    0.105    0.000 